In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import luigi

In [2]:
def postgres_engine():
    db_username = 'postgres'
    db_password = 'qwerty123'
    db_host = 'localhost:5432'
    db_name = 'intro_db'

    engine_str = f"postgresql://{db_username}:{db_password}@{db_host}/{db_name}"
    engine = create_engine(engine_str)

    return engine

In [19]:
class LoadData(luigi.Task):
    def requires(self):
        pass
    
    def run(self):
        dw_engine = postgres_engine()

        transform_data_db = pd.read_csv('transform-data/sales_clean.csv')

        create_table_query = """
        CREATE TABLE IF NOT EXISTS intro_db (
            created_at TIMESTAMP NOT NULL DEFAULT now(),
            name_product TEXT NOT NULL,
            category TEXT NOT NULL,
            link TEXT NOT NULL,
            rating NUMERIC NOT NULL,
            rating_count INT NOT NULL,
            price_sale_rupee NUMERIC NOT NULL,
            discount TEXT NOT NULL
        );
        """

        with dw_engine.connect() as conn:
            conn.execute(text(create_table_query))

        transform_data_db.to_sql(name="intro_db", con=dw_engine, if_exists="append", index=False)


    def output(self):
        return luigi.LocalTarget('raw-data-2/percobaan_db.csv')

In [13]:
class LoadData(luigi.Task):
    def requires(self):
        pass
    
    def run(self):
        dw_engine = postgres_engine()

        # Membaca data dari CSV
        transform_data_db = pd.read_csv('transform-data/sales_clean.csv')

        # Query untuk membuat tabel jika belum ada
        create_table_query = """
        CREATE TABLE IF NOT EXISTS intro_db (
            name_product TEXT NOT NULL,
            category TEXT NOT NULL,
            link TEXT NOT NULL,
            rating TEXT NOT NULL,
            rating_count TEXT NOT NULL,
            price_sale_rupee NUMERIC NOT NULL,
            discount TEXT NOT NULL,
            created_at TIMESTAMP NOT NULL DEFAULT now()
        );
        """

        with dw_engine.connect() as conn:
            # Membuat tabel jika belum ada
            conn.execute(text(create_table_query))

            # Memasukkan data dengan kolom-kolom tertentu
            for _, row in transform_data_db.iterrows():
                insert_query = """
                INSERT INTO intro_db (name_product, category, link, rating, rating_count, price_sale_rupee, discount)
                VALUES (:name_product, :category, :link, :rating, :rating_count, :price_sale_rupee, :discount);
                """
                conn.execute(
                    text(insert_query),
                    {
                        "name_product": row["name_product"],
                        "category": row["category"],
                        "link": row["link"],
                        "rating": row["rating"],
                        "rating_count": row["rating_count"],
                        "price_sale_rupee": row["price_sale_rupee"],
                        "discount": row["discount"],
                    },
                )

    def output(self):
        return luigi.LocalTarget('raw-data-2/percobaan_db.csv')


In [20]:
luigi.build([LoadData()], local_scheduler = True)

DEBUG: Checking if LoadData() is complete
INFO: Informed scheduler that task   LoadData__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 18644] Worker Worker(salt=7576695779, workers=1, host=zueible, username=LENOVO, pid=18644) running   LoadData()


INFO: [pid 18644] Worker Worker(salt=7576695779, workers=1, host=zueible, username=LENOVO, pid=18644) done      LoadData()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   LoadData__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=7576695779, workers=1, host=zueible, username=LENOVO, pid=18644) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 ran successfully:
    - 1 LoadData()

This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



True